# How to Build a Video Segment Copy Detection System

In the previous example we have demonstrated how to use milvus and towhee to build a simple video deduplication system at video level. In this tutorial, we will demonstrate video duplication detection down to segment level. 

**What's the differents between video-level deduplication and segment-level deduplication?**


- Video-level deduplication is a method for situations with high repetition. It finds duplicate videos by comparing the similarity between the embeddings of the whole video. Since only one embedding is extracted from a video, this method works faster. But the limitation of this method is also obvious: it is not good for detecting similar videos of different lengths. For example, the first quarter of video A and video B are exactly the same, but their embeddings may not be similar. In this case, it is obviously impossible to detect infringing content.
 
- Segment-level deduplication detects the specific start and end times of repeated segments, which can handle complex clipping and insertion of video segments as well as situations where the video lengths are not equal. It does so by comparing the similarity between video frames. Obviously, we need to use this method in the actual task of mass video duplication checking. Of course, the speed of this method will be slower than the one of video level.

**What are Milvus & Towhee?**

- Milvus is the most advanced open-source vector database built for AI applications and supports nearest neighbor embedding search across tens of millions of entries.
- Towhee is a framework that provides ETL for unstructured data using SoTA machine learning models.

In this tutorial, we will demonstrate video duplication detection at segment level using Towhee and Milvus. Moreover, we managed to make the core functionality as simple as few lines of code, with which you can start hacking your own video deduplication engine.


## Preparation

### Install packages

Make sure you have installed required python packages:

| package |
| -- |
| pymilvus |
| towhee |
| pillow |
| ipython |
| numpy |
| plyvel |

In [1]:
! python -m pip install -q pymilvus towhee pillow ipython numpy plyvel

### Prepare the data

First, we need to prepare the dataset and Milvus environment.   

The VCDB core dataset is almost all full video length repetition, which is not suitable for the evaluation of segment repetition detection technology. In this tutorial, we use [VCSL dataset](https://arxiv.org/abs/2203.02654).

VCSL is a large-scale real dataset for video duplication detection from Youtube and Bilibili. Unlike VCDB, its cheating changes to video frames are more complex, including crop, filter, text overlay, background, cam-cording, picture in picture, even recent deepfake, etc. there are a wide range of content transformations among over 280k segment copies in VCSL, and these realistic skillful transformations bring great challenges to segment-level copy detection. 

In this tutorial, we use only a mini set of VCSL. It contains 5 events with three videos in each event. There is also a broken video for robustness testing.

In [2]:
! curl -L https://github.com/towhee-io/examples/releases/download/data/VCSL-demo.zip -O
! unzip -q -o VCSL-demo.zip

In [3]:
! tree ./VCSL-demo/

./VCSL-demo/
├── baisuishan
│   ├── 03584e404c0847fcbe5f9c486e8f8fc7-【宇哥】原来百岁山的广告是这个意思！-1UE411V74D.flv
│   ├── 217a12c936414660a53b55b22e2aea59-20200607百岁山-1kf4y127GB.flv
│   └── 41c4eaced0d24ebba50d180026531025-廣東有線翡翠台「瞬間看地球」+插播「百歲山」廣告（錄影時間：2019年12月23日 上午8時55分）-1UJ411s7we.flv
├── crashed_video
│   └── Clip46_KingKong_MP4_Sorenson_750K_720x404_30fps_16x9_MPAA.mp4
├── donghua
│   ├── 043bee7a71f347f18e8576bb1a01c86b-Langrisser M 'Dream Simulation' OPレオンが戻ってきた-y3qAPvWnL18.mkv
│   ├── 1134884a6c544a00b3bce99457a2a98d-LANGRISSER Mobile OP-EA4PmSWGr8o.mp4
│   └── 1b4e048011714eab928650f64e6370a6-123-JqSBkvPsYBw.mkv
├── huangpi
│   ├── 01414786cbe74d2d95e2b71df26f39a3-【外星金轮】金轮....已经无所谓了  (反正马老师看不到)-1sh411B7g1.flv
│   ├── 0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv
│   └── 9ec6f467472a428e91fd45bb688e3be1-酒醉的广场黄皮外星人-1uU4y1E7n6.flv
├── liulangdiqiu
│   ├── d62ce5becff14a0c9c7dab5eea6647dc-《流浪地球》吴京成青少年偶像，新闻联播盛赞：中国人的浪漫科幻-1qf4y1G7gM.flv
│   ├── e5dc80abd7a24b47accde190c9fdbcdc-【新

Define some helper function to convert video to gif so that we can have a look at these videos.   

In [4]:
from IPython import display
from pathlib import Path
import towhee
from PIL import Image
import os

def display_gif(video_path_list, text_list):
    html = ''
    for video_path, text in zip(video_path_list, text_list):
        html_line = '<img src="{}"> {} <br/><br/>'.format(video_path, text)
        html += html_line
    return display.HTML(html)

    
def convert_video2gif(video_path, output_gif_path, num_samples=16):
    frames = (
        towhee.glob(video_path)
              .video_decode.ffmpeg(start_time=0.0, end_time=1000.0, sample_type='time_step_sample', args={'time_step': 8})
              .to_list()[0]
    )
    imgs = [Image.fromarray(frame) for frame in frames]
    imgs = [img.resize((int(img.width/4), int(img.height/4)), Image.NEAREST) for img in imgs]
    imgs[0].save(fp=output_gif_path, format='GIF', append_images=imgs[1:], save_all=True, loop=0)


def display_gifs_from_video(video_path_list, text_list, tmpdirname = './tmp_gifs'):
    Path(tmpdirname).mkdir(exist_ok=True)
    gif_path_list = []
    for video_path in video_path_list:
        video_name = str(Path(video_path).name).split('.')[0]
        gif_path = Path(tmpdirname) / (video_name + '.gif')
        convert_video2gif(video_path, gif_path)
        gif_path_list.append(gif_path)
    return display_gif(gif_path_list, text_list)

In [5]:
import random
vcsl_demo_root = './VCSL-demo/'
all_file_pattern = os.path.join(vcsl_demo_root, '*', '*.*')

event_list = os.listdir(vcsl_demo_root)
if 'crashed_video' in event_list:
    event_list.remove('crashed_video')

random_event = random.choice(event_list)
random_event_folder = os.path.join(vcsl_demo_root, random_event)
random_event_videos = [os.path.join(random_event_folder, video_file) for video_file in os.listdir(random_event_folder)]
tmpdirname = './tmp_gifs'
display_gifs_from_video(random_event_videos, random_event_videos, tmpdirname=tmpdirname)

In [6]:
import towhee
import numpy as np
from towhee.types import Image


os.environ["CUDA_VISIBLE_DEVICES"] = '1'


@towhee.register
def get_image(x):
	for i in x:
		yield Image(i.__array__(), 'RGB')

@towhee.register
def merge_ndarray(x):
	return np.concatenate(x).reshape(-1, x[0].shape[0])

@towhee.register
def split_res(x):
	return [i.path for i in x], [i.score for i in x]

### Create a Milvus Collection

Before getting started, please make sure you have [installed milvus](https://milvus.io/docs/v2.1.x/install_standalone-docker.md). Let's first create a `video_deduplication` collection that uses the [L2 distance metric](https://milvus.io/docs/v2.1.x/metric.md#Euclidean-distance-L2) and an [IVF_FLAT index](https://milvus.io/docs/v2.1.x/index.md#IVF_FLAT).

In [7]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
        FieldSchema(name='id', dtype=DataType.INT64, descrition='id of the embedding', is_primary=True, auto_id=True),
        FieldSchema(name='path', dtype=DataType.VARCHAR,  descrition='path of the embedding',max_length=500),
        FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='video embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='video dedup')
    collection = Collection(name=collection_name, schema=schema)

    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":1}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

collection = create_milvus_collection('video_deduplication', 256)


## Video Copy Detection

In this section, we'll show how to build our Video Copy Detection engine using Milvus. The basic idea behind Video Copy Detection is the extract embeddings from videos using Deep Neural Network and store them in Milvus, then get query videos embeddings and compare with those stored in Milvus.

We use [Towhee](https://towhee.io/), a machine learning framework that allows for creating data processing pipelines. [Towhee](https://towhee.io/) also provides predefined operators which implement insert and query operation in Milvus.


### Load Video Embeddings into Milvus

For every video, we decode it to image frames, and then using neural network to extract their embeddings. We insert them to Milvus and levelDB for storage.
![](video_decopy_insert.png)


In [8]:
%%time
dc = (
	towhee.glob['url'](all_file_pattern).stream()
		.video_decode.ffmpeg['url', 'frames'](sample_type='time_step_sample', args={'time_step': 1})
		.get_image['frames', 'images']()
		.flatten('images')
		.drop_empty()
		.image_embedding.isc['images', 'embeddings']()
		.select['url', 'embeddings']()
		.ann_insert.milvus[('url', 'embeddings'), 'insert_result'](uri='tcp://localhost:19530/video_deduplication')
		.group_by('url')
		.merge_ndarray['embeddings', 'video_embedding']()
		.insert_leveldb[('url', 'video_embedding'), ]('url_vec.db')
		.select['url', 'video_embedding']()
		.show(limit=20)
)

2022-09-28 12:40:40,403 - 140452136294208 - video_decoder.py-video_decoder:120 - ERROR: header damaged


url,video_embedding
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)"
./VCSL-demo/huangpi/01414786cbe74d2d95e2b71df26f39a3-【外星金轮】金轮....已经无所谓了 (反正马老师看不到)-1sh411B7g1.flv,"[-0.03947173, -0.010356861, -0.08993856, ...] shape=(90, 256)"
./VCSL-demo/huangpi/9ec6f467472a428e91fd45bb688e3be1-酒醉的广场黄皮外星人-1uU4y1E7n6.flv,"[0.009808735, -0.016826384, 0.06355846, ...] shape=(59, 256)"
./VCSL-demo/baisuishan/41c4eaced0d24ebba50d180026531025-廣東有線翡翠台「瞬間看地球」+插播「百歲山」廣告（錄影時間：2019年12月23日 上午8時55分）-1UJ411s7we.flv,"[-0.020499738, -0.105837084, 0.07808163, ...] shape=(281, 256)"
./VCSL-demo/baisuishan/217a12c936414660a53b55b22e2aea59-20200607百岁山-1kf4y127GB.flv,"[0.02334698, -0.022769544, 0.03606544, ...] shape=(61, 256)"
./VCSL-demo/baisuishan/03584e404c0847fcbe5f9c486e8f8fc7-【宇哥】原来百岁山的广告是这个意思！-1UE411V74D.flv,"[-0.02180116, -0.04651391, 0.031187996, ...] shape=(227, 256)"
./VCSL-demo/liulangdiqiu/d62ce5becff14a0c9c7dab5eea6647dc-《流浪地球》吴京成青少年偶像，新闻联播盛赞：中国人的浪漫科幻-1qf4y1G7gM.flv,"[-0.048483953, -0.08141277, 0.117679514, ...] shape=(103, 256)"
./VCSL-demo/liulangdiqiu/ef65e0f662e646a88a13b6eddb640e48-《流浪地球》上《新闻联播》排面鸭！CCTV央视给力！-1xb411U7uE.flv,"[0.07897866, 0.09988424, 0.077609695, ...] shape=(198, 256)"
./VCSL-demo/liulangdiqiu/e5dc80abd7a24b47accde190c9fdbcdc-【新闻联播】流浪地球上央视一套7点档啦-1db411m7f8.flv,"[-0.0009121081, 0.0041335938, -0.011865338, ...] shape=(33, 256)"
./VCSL-demo/madongmei/8ad81fc9fe0a47dbaab1b4cdc40bf07b-行了，大爷你一边凉快去吧，神曲《马冬梅》-1t54y117JK.flv,"[-0.058290888, -0.082258746, -0.015635852, ...] shape=(84, 256)"


CPU times: user 4h 9min 11s, sys: 5min 46s, total: 4h 14min 57s
Wall time: 4min 18s


If you see "ERROR: header damaged", it's because the sample dataset already has a corrupt video and we skip it. This is to simulate that in practice, where there may be a small amount of corrupted video in the huge video data, but they should not affect the execution of other normal video.

### Query videos

In theory, for each query video, it is necessary to match and retrieve all the videos in the database, which will cause huge overhead. In this tutorial, we perform a rough selection to solve this problem. 
 
First, for every query frame, we retrieve a certain number of similar frames through Milvus, which match for a specified video. The videos of these frames are then aggregated, sorted, and filtered. Then, the video embeddings of remaining videos and embedding of the query video are processed for localizing copyed segments. In this way, we can filter out videos with low similarity, saving a lot of computation for the whole pipeline.
![](video_decopy_query.png)

In [9]:
%%time
collection.load()
dc = (
	towhee.glob['url'](all_file_pattern).stream()
		.video_decode.ffmpeg['url', 'frames'](sample_type='time_step_sample', args={'time_step': 1})
		.get_image['frames', 'images']()
		.flatten('images')
		.drop_empty()
		.image_embedding.isc['images', 'embeddings']()
		.select['url', 'embeddings']()
		.ann_search.milvus['embeddings', 'results'](collection=collection, limit=64, output_fields=['path'], metric_type='IP')
		.split_res['results', ('retrieved_urls','scores')]()
		.group_by('url')
		.video_copy_detection.select_video[('retrieved_urls','scores'), 'res'](top_k=8, reduce_function='sum', reverse=True)
		.from_leveldb['res', 'retrieved_embedding']('url_vec.db', True)
		.merge_ndarray['embeddings', 'video_embedding']()
		.flatten('retrieved_embedding', 'res')
		.video_copy_detection.temporal_network[('video_embedding', 'retrieved_embedding'), ('range', 'range_score')]()
		.select['url', 'video_embedding',  'res', 'retrieved_embedding', 'range', 'range_score']()
		.show(limit=100)
)

url,video_embedding,res,retrieved_embedding,range,range_score
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)",./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)","[[0, 0, 117, 117]] len=1",[1.0085470141508641] len=1
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)",./VCSL-demo/huangpi/01414786cbe74d2d95e2b71df26f39a3-【外星金轮】金轮....已经无所谓了 (反正马老师看不到)-1sh411B7g1.flv,"[-0.03947173, -0.010356861, -0.08993856, ...] shape=(90, 256)","[[0, 2, 117, 87]] len=1",[0.4509598996379588] len=1
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)",./VCSL-demo/huangpi/9ec6f467472a428e91fd45bb688e3be1-酒醉的广场黄皮外星人-1uU4y1E7n6.flv,"[0.009808735, -0.016826384, 0.06355846, ...] shape=(59, 256)","[[52, 0, 116, 58]] len=1",[0.47772424240581324] len=1
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)",./VCSL-demo/baisuishan/41c4eaced0d24ebba50d180026531025-廣東有線翡翠台「瞬間看地球」+插播「百歲山」廣告（錄影時間：2019年12月23日 上午8時55分）-1UJ411s7we.flv,"[-0.020499738, -0.105837084, 0.07808163, ...] shape=(281, 256)",,
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)",./VCSL-demo/donghua/043bee7a71f347f18e8576bb1a01c86b-Langrisser M 'Dream Simulation' OPレオンが戻ってきた-y3qAPvWnL18.mkv,"[-0.03859018, -0.00797931, -0.11145613, ...] shape=(196, 256)",,
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)",./VCSL-demo/baisuishan/03584e404c0847fcbe5f9c486e8f8fc7-【宇哥】原来百岁山的广告是这个意思！-1UE411V74D.flv,"[-0.02180116, -0.04651391, 0.031187996, ...] shape=(227, 256)",,
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)",./VCSL-demo/baisuishan/217a12c936414660a53b55b22e2aea59-20200607百岁山-1kf4y127GB.flv,"[0.02334698, -0.022769544, 0.03606544, ...] shape=(61, 256)",,
./VCSL-demo/huangpi/0fba4debd48245699971c4c18608cde3-黄皮外星人 低成本大制作-1vP4y1s79h.flv,"[-0.020633303, -0.056049056, -0.021473853, ...] shape=(118, 256)",./VCSL-demo/donghua/1b4e048011714eab928650f64e6370a6-123-JqSBkvPsYBw.mkv,"[-0.03056283, 0.067672186, -0.03823933, ...] shape=(89, 256)",,
./VCSL-demo/huangpi/01414786cbe74d2d95e2b71df26f39a3-【外星金轮】金轮....已经无所谓了 (反正马老师看不到)-1sh411B7g1.flv,"[-0.03947173, -0.010356861, -0.08993856, ...] shape=(90, 256)",./VCSL-demo/huangpi/01414786cbe74d2d95e2b71df26f39a3-【外星金轮】金轮....已经无所谓了 (反正马老师看不到)-1sh411B7g1.flv,"[-0.03947173, -0.010356861, -0.08993856, ...] shape=(90, 256)","[[0, 0, 89, 89]] len=1",[1.0112359369738717] len=1
./VCSL-demo/huangpi/01414786cbe74d2d95e2b71df26f39a3-【外星金轮】金轮....已经无所谓了 (反正马老师看不到)-1sh411B7g1.flv,"[-0.03947173, -0.010356861, -0.08993856, ...] shape=(90, 256)",./VCSL-demo/huangpi/9ec6f467472a428e91fd45bb688e3be1-酒醉的广场黄皮外星人-1uU4y1E7n6.flv,"[0.009808735, -0.016826384, 0.06355846, ...] shape=(59, 256)","[[27, 1, 83, 58]] len=1",[0.7184088367276487] len=1


CPU times: user 5h 8min 21s, sys: 7min 21s, total: 5h 15min 42s
Wall time: 5min 32s


For each frame of each query video, we query the most similar frame information of 64 frames from Milvus. We aggregate and sort this information, and select candidate videos with topk=8. For this query video and the corresponding 8 candidate videos, `temporal_network` calculation is performed, and finally the detected duplicate segments are obtained. 

Note that our query uses the same dataset, in which there are 5 events, and each event has 3 videos, which are copies of each other. Using this dataset to query itself means that for each video, the correct query result should be the three videos under its own event. 

The output `range` column is detected segments list, which format is list of `[query_start_second, ref_start_second, query_end_second, ref_end_second]`. We can observe that each query video does detect only 3 results of its own event, which is consistent with ground truth.
 

In [10]:
# import shutil
# shutil.rmtree(tmpdirname)